In [1]:
#%% Importing libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as cb
from datetime import datetime

In [2]:
#Importing datasets
train_data = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/train.csv")
cust_demo_data = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/customer_demographics.csv")
campaign_data = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/campaign_data.csv")
coupon_item_mapping = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/coupon_item_mapping.csv")
customer_transaction_data = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/customer_transaction_data.csv")
item_data = pd.read_csv("E:/LBSIM/Trimester 5/TP/Data/item_data.csv")

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 5 columns):
id                   78369 non-null int64
campaign_id          78369 non-null int64
coupon_id            78369 non-null int64
customer_id          78369 non-null int64
redemption_status    78369 non-null int64
dtypes: int64(5)
memory usage: 3.0 MB


In [3]:
#Merging datasets
train_data_merge = pd.merge(train_data,campaign_data,how="inner",on="campaign_id")
train_data_merge.info()

train_data_merge = pd.merge(train_data_merge,coupon_item_mapping,how="inner",on="coupon_id")
train_data_merge.info()

train_data_merge = pd.merge(train_data_merge,item_data,how="inner",on="item_id")
train_data_merge.info()

train_data_merge = pd.merge(train_data_merge,cust_demo_data,how="left",on="customer_id")
train_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78369 entries, 0 to 78368
Data columns (total 8 columns):
id                   78369 non-null int64
campaign_id          78369 non-null int64
coupon_id            78369 non-null int64
customer_id          78369 non-null int64
redemption_status    78369 non-null int64
campaign_type        78369 non-null object
start_date           78369 non-null object
end_date             78369 non-null object
dtypes: int64(5), object(3)
memory usage: 5.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6420694 entries, 0 to 6420693
Data columns (total 9 columns):
id                   int64
campaign_id          int64
coupon_id            int64
customer_id          int64
redemption_status    int64
campaign_type        object
start_date           object
end_date             object
item_id              int64
dtypes: int64(6), object(3)
memory usage: 489.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6420694 entries, 0 to 6420693
Data columns 

In [4]:
#%% Keeping unique values and droping duplicates 
pd.unique(train_data_merge[['id','campaign_id','coupon_id',
                            'customer_id','redemption_status','item_id']].values.ravel())

customer_transaction_data.info()
train_data_merge = pd.merge(train_data_merge,customer_transaction_data,how="left",on=['customer_id','item_id'])
train_data_merge.info()
train_data_merge.isnull().sum()

train_data_merge.drop_duplicates(keep="first", inplace=True)
train_data_merge.info()
train_data_merge.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
date               1048575 non-null object
customer_id        1048575 non-null int64
item_id            1048575 non-null int64
quantity           1048575 non-null int64
selling_price      1048575 non-null float64
other_discount     1048575 non-null float64
coupon_discount    1048575 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 56.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6469802 entries, 0 to 6469801
Data columns (total 23 columns):
id                   int64
campaign_id          int64
coupon_id            int64
customer_id          int64
redemption_status    int64
campaign_type        object
start_date           object
end_date             object
item_id              int64
brand                int64
brand_type           object
category             object
age_range            object
marital_status       object
rented               float

id                         0
campaign_id                0
coupon_id                  0
customer_id                0
redemption_status          0
campaign_type              0
start_date                 0
end_date                   0
item_id                    0
brand                      0
brand_type                 0
category                   0
age_range            2627401
marital_status       2627401
rented               2627401
family_size          2627401
no_of_children       2627401
income_bracket       2627401
date                 6357597
quantity             6357597
selling_price        6357597
other_discount       6357597
coupon_discount      6357597
dtype: int64

In [ ]:
#%% Replacing null values
train_data_merge['selling_price'].mean(skipna=True)
train_data_merge['coupon_discount'].mean(skipna=True)
train_data_merge['other_discount'].mean(skipna=True)
train_data_merge['quantity'].mode(dropna=True)
train_data_merge['rented'].mode(dropna=True)
train_data_merge['income_bracket'].mode(dropna=True)
train_data_merge['no_of_children'].mode(dropna=True)
train_data_merge['family_size'].mode(dropna=True)
train_data_merge['age_range'].mode(dropna=True)

train_data_merge['other_discount'].fillna(train_data_merge['other_discount'].mean(skipna=True),inplace=True)
train_data_merge['coupon_discount'].fillna(train_data_merge['coupon_discount'].mean(skipna=True),inplace=True)
train_data_merge['selling_price'].fillna(train_data_merge['selling_price'].mean(skipna=True),inplace=True)
train_data_merge['quantity'].fillna(1,inplace=True)
train_data_merge['income_bracket'].fillna(5,inplace=True)
train_data_merge['no_of_children'].fillna(0,inplace=True)
train_data_merge['family_size'].fillna(2,inplace=True)
train_data_merge['marital_status'].fillna('Married',inplace=True)
train_data_merge['rented'].fillna(0,inplace=True)
train_data_merge['age_range'].fillna('46-55',inplace=True)

train_data_merge.info()
train_data_merge.isnull().sum()

In [5]:
#%% Obtaining relevant columns
train_data_merge.columns
columns = ['campaign_type','brand_type', 'category','brand',
           'marital_status','rented','income_bracket','age_range','no_of_children',
           'selling_price','other_discount', 'coupon_discount','quantity']
x_data = train_data_merge[columns]
y_data = train_data_merge['redemption_status']
x_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6469210 entries, 0 to 6469801
Data columns (total 13 columns):
campaign_type      object
brand_type         object
category           object
brand              int64
marital_status     object
rented             float64
income_bracket     float64
age_range          object
no_of_children     float64
selling_price      float64
other_discount     float64
coupon_discount    float64
quantity           float64
dtypes: float64(7), int64(1), object(5)
memory usage: 691.0+ MB
